In [1]:
import numpy as np

from scipy.constants import e, c

from scipy.constants import physical_constants

import os

In [2]:
from cpymad.madx import Madx

import sixtracklib as stl
import pysixtrack

# Versioning

## SixTrackLib

In [3]:
f = os.path.dirname(stl.__file__)
f

'/home/oeftiger/gsi/git/sixtracklib_aoeftiger/python/sixtracklib'

In [4]:
!cd $f && git log | head -4

commit f42c420da20602c9c4c08172338ed3019bb20d25
Merge: deff9373 20f49532
Author: Adrian Oeftiger <a.oeftiger@gsi.de>
Date:   Wed Jan 8 14:52:08 2020 +0100


## PySixTrack

In [5]:
f = os.path.dirname(pysixtrack.__file__)
f

'/home/oeftiger/gsi/git/pysixtrack/pysixtrack'

In [6]:
!cd $f && git log | head -4

commit a1c73d9578a20e404dff54e1330aebe77eb763d5
Merge: 1d8f5db 4ce2927
Author: Riccardo De Maria <riccardodemaria@gmail.com>
Date:   Wed Jan 8 17:48:29 2020 +0100


# Parameters

In [7]:
nmass = physical_constants['atomic mass constant energy equivalent in MeV'][0] * 1e-3
# nmass = 0.931494061 # MAD-X value

In [8]:
A = 238
Q = 28

Ekin_per_nucleon = 0.2e9 # in eV

###

mass = A * nmass * 1e9 * e / c**2 # in kg
charge = Q * e # in Coul

Ekin = Ekin_per_nucleon * A
p0c = np.sqrt(Ekin**2 + 2*Ekin*mass/e * c**2) # in eV

Etot = np.sqrt(p0c**2 + (mass/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e # in SI units
gamma = np.sqrt(1 + (p0 / (mass * c))**2)
beta = np.sqrt(1 - gamma**-2)

## MAD-X setup

In [9]:
madx = Madx()
madx.options.echo = False
madx.options.warn = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2020.01.13 18:19:46      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [10]:
madx.input('''SET, format="22.14e";''')

True

In [11]:
madx.input('''
qf: multipole, knl={0, 0.3};

testing: sequence, l = 1;
qf, at = 0.5;
endsequence;
''')

True

In [12]:
madx.command.beam(
    particle='ion', mass=A*nmass, charge=Q, energy=Etot)

True

In [13]:
madx.use(sequence='testing')

In [14]:
twiss_no_error = madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.31720858935157e-01   0.00000000000000e+00   1.57812500000000e+00   0.00000000000000e+00 

                 dxrms                 xcomax                 xcorms                     q2 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   1.19637257107869e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   2.47812500000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                ycomax                 ycorms                 deltap                synch_1 
  0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

In [15]:
q1_no_error = twiss_no_error.summary.q1

In [16]:
madx.select(flag='error', pattern='qf')

True

In [17]:
madx.command.efcomp(
    dkn=[0, 1e-2, 5.006767857e-06, 0.01309999342, -0.1972873115, -73.9750876, -9.804122175],
    dks=[0.0, -1.315727925e-05, 1.819912072e-05, -0.002619217458, 0.1995854548, 21.55489111, -919.2177798],
)

Assigned field errors to 1 elements


True

In [18]:
madx.command.ealign(dx=1e-3, dy=-1e-3, dpsi=0.1)

Assigned alignment errors to 1 elements


True

In [19]:
# qf = madx.sequence.testing.elements[1] 
### --> only "elements" does not work!! need "expanded_elements"
qf = madx.sequence.testing.expanded_elements[2]
qf

qf: multipole, at=0.5, knl={0.0,0.3};

In [20]:
qf.field_errors.dkn, qf.field_errors.dks

([0.0,
  0.01,
  5.006767857e-06,
  0.01309999342,
  -0.1972873115,
  -73.9750876,
  -9.804122175,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  -1.315727925e-05,
  1.819912072e-05,
  -0.002619217458,
  0.1995854548,
  21.55489111,
  -919.2177798,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0])

In [21]:
qf.align_errors.dx

0.001

In [22]:
qf.align_errors.dy

-0.001

In [23]:
qf.align_errors.dpsi

0.1

In [24]:
twiss_with_error = madx.twiss(betx=1, bety=1, dx=0, dy=0);

enter Twiss module

++++++ table: summ

                length                 orbit5                   alfa                gammatr 
  1.00000000000000e+00  -0.00000000000000e+00   0.00000000000000e+00   0.00000000000000e+00 

                    q1                    dq1                betxmax                  dxmax 
  1.31817273736839e-01   0.00000000000000e+00   1.57311844149584e+00   2.13336247602740e-04 

                 dxrms                 xcomax                 xcorms                     q2 
  1.34925689979659e-04   1.21108807760132e-04   7.65959354458981e-05   1.19576035085275e-01 

                   dq2                betymax                  dymax                  dyrms 
  0.00000000000000e+00   2.48457241597610e+00   3.21846909795632e-04   2.03553858568191e-04 

                ycomax                 ycorms                 deltap                synch_1 
  1.82709201856848e-04   1.15555445467821e-04   0.00000000000000e+00   0.00000000000000e+00 

               synch_2   

In [25]:
q1_with_error = twiss_with_error.summary.q1

$\implies$ **The tune has changed with the additional quadrupole error:**

In [26]:
1 - q1_no_error / q1_with_error

0.0007314276721764879

## MAD-X tracking

In [27]:
madx.input('''
TRACK, onepass, onetable, file=output.;

START, x=1e-03, px=1e-06, y=-1e-03, py=-5e-07, t=0.1, pt=3e-03;
!START, x=0e-03, px=0e-06, y=0e-03, py=0e-07, t=0.1, pt=3e-03;

RUN, turns=1;

ENDTRACK;
''')

enter TRACK module
one pass is on

++++++ table: tracksumm

    number       turn                      x                     px 
         1          0   1.00000000000000e-03   1.00000000000000e-06 
         1          1   1.00092721250931e-03   8.64207412287082e-07 

                     y                     py                      t                     pt 
 -1.00000000000000e-03  -5.00000000000000e-07   1.00000000000000e-01   2.99999999999989e-03 
 -1.00055019488585e-03  -6.06194506773380e-07   1.06259314454161e-01   2.99999999999989e-03 

                     s                      e 
  0.00000000000000e+00   0.00000000000000e+00 
  1.00000000000000e+00   0.00000000000000e+00 
exit TRACK module



True

In [28]:
headers = list(np.genfromtxt(
    "output.one", skip_header=51, max_rows=1, dtype=str)[1:])

initial_distribution_madx = np.genfromtxt(
    "output.one", skip_header=54, max_rows=1, dtype=np.float64)

final_distribution_madx = np.genfromtxt(
    "output.one", skip_header=55, max_rows=1, dtype=np.float64)

In [29]:
def get_betai(PT, mass=mass, p0=p0):
    restmass = mass * c**2
    restmass_sq = restmass**2
    E0 = np.sqrt((p0 * c)**2 + restmass_sq)
    
    E = E0 + PT * p0 * c
    gammai = E / restmass
    betai = np.sqrt(1 - 1. / (gammai * gammai))
    return betai

In [30]:
pt_madx = final_distribution_madx[headers.index('PT')]
betai = get_betai(pt_madx)

x_madx = final_distribution_madx[headers.index('X')]
px_madx = final_distribution_madx[headers.index('PX')]
y_madx = final_distribution_madx[headers.index('Y')]
py_madx = final_distribution_madx[headers.index('PY')]
zeta_madx = final_distribution_madx[headers.index('T')] * betai
delta_madx = np.sqrt(1 + 2 * pt_madx / beta + pt_madx**2) - 1

## SixTrackLib setup

In [31]:
madx.esave(file='errors')
madx.command.readtable(file='errors', table='errors')
errors = madx.table.errors

Want to make named table: errors


In [32]:
# PySixTrack, lattice transfer and preparation!

pysixtrack_elements = pysixtrack.Line.from_madx_sequence(
    madx.sequence.testing, exact_drift=True,
)

pysixtrack_elements.apply_madx_errors(errors)

pysixtrack_elements.remove_zero_length_drifts(inplace=True);
pysixtrack_elements.merge_consecutive_drifts(inplace=True);

The PySixTrack multipole has a quadrupole component consisting of the sum of the MAD-X original quadrupole component (0.3) + the error (0.01):

In [33]:
pysixtrack_elements.elements[3].knl

[0.0,
 0.31,
 5.006767857e-06,
 0.01309999342,
 -0.1972873115,
 -73.9750876,
 -9.804122175]

In [34]:
pysixtrack_elements.elements[1].dx

0.001

In [35]:
pysixtrack_elements.elements[1].dy

-0.001

In [36]:
pysixtrack_elements.elements[2].angle

5.729577951308232

In [37]:
elements = stl.Elements.from_line(pysixtrack_elements)
elements.BeamMonitor(num_stores=1);

In [38]:
pysixtrack_particles = pysixtrack.Particles.from_madx_track(madx)

In [39]:
# necessary to transfer to SixTrackLib with this version of PySixTrack
pysixtrack_particles.state = np.array([1, 0])
pysixtrack_particles.elemid = 0

pysixtrack_particles.remove_lost_particles()

In [40]:
particles = stl.ParticlesSet().Particles(num_particles=1)

particles.from_pysixtrack(pysixtrack_particles, particle_index=0)

In [41]:
trackjob = stl.TrackJob(elements, particles) #, device="opencl:0.0")

## SixTrackLib tracking

In [42]:
trackjob.track_until(1)
trackjob.collect()

In [43]:
x = trackjob.output.particles[0].x[0]
px = trackjob.output.particles[0].px[0]
y = trackjob.output.particles[0].y[0]
py = trackjob.output.particles[0].py[0]
zeta = trackjob.output.particles[0].zeta[0]
delta = trackjob.output.particles[0].delta[0]

# evaluation SixTrackLib vs. MAD-X:

In [44]:
print ('x error in %: {}'.format(100 * (x - x_madx) / x))

x error in %: -2.5996747630048217e-13


In [45]:
print ('px error in %: {}'.format(100 * (px - px_madx) / px))

px error in %: 1.5436998953699485e-12


In [46]:
print ('y error in %: {}'.format(100 * (y - y_madx) / y))

y error in %: 1.7337695648289925e-13


In [47]:
print ('py error in %: {}'.format(100 * (py - py_madx) / py))

py error in %: 6.46249238037989e-13


In [48]:
print ('zeta error in %: {}'.format(100 * (zeta - zeta_madx) / zeta))

zeta error in %: -1.2608368628817337e-13


In [49]:
print ('delta error in %: {}'.format(100 * (delta - delta_madx) / delta))

delta error in %: 0.0


In [50]:
delta == delta_madx

True